In [18]:
#prerequisites
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble  import RandomForestRegressor
from sklearn.metrics import mean_squared_error ,r2_score
data = sns.load_dataset("car_crashes")
df = pd.DataFrame(data)
df
df=df.drop(columns=["abbrev"],axis=1)

df

,total,speeding,alcohol,not_distracted,no_previous,ins_premium,ins_losses
0,18.8,7.332,5.640,18.048,15.040,784.55,145.08
1,18.1,7.421,4.525,16.290,17.014,1053.48,133.93
2,18.6,6.510,5.208,15.624,17.856,899.47,110.35
3,22.4,4.032,5.824,21.056,21.280,827.34,142.39
4,12.0,4.200,3.360,10.920,10.680,878.41,165.63
5,13.6,5.032,3.808,10.744,12.920,835.50,139.91
6,10.8,4.968,3.888,9.396,8.856,1068.73,167.02
7,16.2,6.156,4.860,14.094,16.038,1137.87,151.48
8,5.9,2.006,1.593,5.900,5.900,1273.89,136.05
9,17.9,3.759,5.191,16.468,16.826,1160.13,144.18


In [19]:
#via IQR
Q1 = df["speeding"].quantile(0.25)
Q3 = df["speeding"].quantile(0.75) 
IQR =  Q3 -Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR 
outliers = df[(df["speeding"] < lower) | (df["speeding"] > upper)]
print("Outliers : ",outliers)

Outliers :  Empty DataFrame
Columns: [total, speeding, alcohol, not_distracted, no_previous, ins_premium, ins_losses]
Index: []


In [21]:
#removing outliers 
df=df[(df["speeding"] >= lower) & (df["speeding"] <= upper)]


In [22]:
 #model preparation
x = df.drop("total",axis=1)
y = df["total"]

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)
linearRegressionModel = LinearRegression()


In [23]:
#test the model 
linearRegressionModel.fit(x_train,y_train)

LinearRegression()

In [24]:
#make predictions
y_pred= linearRegressionModel.predict(x_test)

In [25]:
#evaluate the results of the model 
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
r2=r2_score(y_test,y_pred)
print("RMSE SCORE OF LINEAR REGRESSION MODEL : ",rmse)
print("R2 SCORE OF LINEAR REGRESSION MODEL : ",r2)

RMSE SCORE OF LINEAR REGRESSION MODEL :  1.4785852466056522
R2 SCORE OF LINEAR REGRESSION MODEL :  0.8791208489722776


In [26]:
#Random Forest 
randomForestModel  = RandomForestRegressor(n_estimators=10,random_state=42)
randomForestModel.fit(x_train,y_train)
yy_pred=randomForestModel.predict(x_test)
rmse2 = np.sqrt(mean_squared_error(y_test,yy_pred))
r22 = r2_score(y_test,yy_pred)
print("RMSE SCORE OF Random Forest Regressor Model: ",rmse2)
print("R2 SCORE OF Random Forest Regressor MODEL : ",r22)


RMSE SCORE OF Random Forest Regressor Model:  1.5997016767339614
R2 SCORE OF Random Forest Regressor MODEL :  0.8585064430634257
